# Agente com Autogen - Resolução Colaborativa de Tarefas com Agente de Codificação e Planejamento - Hands-On


Agradecimento ao professor: Júlio Cesar dos Reis

## Descrição

Demonstramos como usar múltiplos agentes para trabalhar juntos e concluir uma tarefa que envolve a busca de informações na web e programação. O AssistantAgent é um agente baseado em LLM capaz de escrever e depurar código Python (em um bloco de código Python) para que um agente usuário possa executá-lo em uma tarefa específica. O UserProxyAgent é um agente que atua como um intermediário para um usuário e executa o código escrito pelo AssistantAgent.
Criamos um agente de planejamento para auxiliar o AssistantAgent. Esse agente de planejamento é uma variação do AssistantAgent baseado em LLM, com uma mensagem de sistema diferente.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install groq
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 651.1/651.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00


In [ ]:
import autogen

import getpass
import os

api_key = os.environ.get("GROQ_API_KEY") or os.getenv("GROQ_API_KEY") or getpass.getpass("Enter your Groq API key: ")

config_list = [
    {
        'model': 'llama-3.3-70b-versatile',
        'api_key': api_key,
        "api_type": "groq"
    },
]


Enter your Groq API key: ··········


## Construção dos Agentes

Construímos o agente de planejamento chamado **"planner"** e um agente proxy de usuário para o planejador chamado **"planner_user"**. No agente proxy de usuário, especificamos `human_input_mode` como **"NEVER"**, garantindo que ele nunca solicite feedback humano.  
Definimos a função `ask_planner` para enviar uma mensagem ao planejador e retornar sua sugestão.

In [ ]:
planner = autogen.AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix.",
)
planner_user = autogen.UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
    code_execution_config={
        "use_docker": False
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]

Construímos o agente assistente e o agente proxy de usuário. No agente proxy de usuário, especificamos human_input_mode como "TERMINATE", o que significa que ele solicitará feedback quando receber um sinal de "TERMINATE" do agente assistente.
Configuramos functions no AssistantAgent e function_map no UserProxyAgent para utilizar a função ask_planner criada.

In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0,
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
        "functions": [
            {
                "name": "ask_planner",
                "description": "ask planner to: 1. get a plan for finishing a task, 2. verify the execution result of the plan and potentially suggest new plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result. The planner does not know the conversation between you and the user, unless you share the conversation with the planner.",
                        },
                    },
                    "required": ["message"],
                },
            },
        ],
    },
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "planning",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    function_map={"ask_planner": ask_planner},
)

## Executar uma Tarefa
Invocamos o método initiate_chat() do agente proxy de usuário para iniciar a conversa. Ao executar a célula abaixo, você será solicitado a fornecer feedback após o agente assistente enviar um sinal "TERMINATE" no final da mensagem. Se você não fornecer nenhum feedback (pressionando Enter diretamente), a conversa será encerrada.

Antes do sinal "TERMINATE", o agente proxy de usuário tentará executar o código sugerido pelo agente assistente em nome do usuário.

In [ ]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Quais são os maiores lagos do mundo?""", #Criar um plano para otimizar a distribuição de vacinas no Brasil, considerando logística e infraestrutura.
)

user_proxy (to assistant):

Quais são os maiores lagos do mundo?

--------------------------------------------------------------------------------
assistant (to user_proxy):

Os maiores lagos do mundo variam de acordo com o critério utilizado, seja pela área superficial, volume de água ou profundidade. Aqui estão alguns dos maiores lagos do mundo, considerando esses diferentes critérios:

1. **Maior lago por área superficial:** O Mar Cáspio, localizado entre a Europa e a Ásia, é o maior lago do mundo em termos de área superficial, cobrindo aproximadamente 371.000 km². No entanto, é importante notar que o Mar Cáspio é tecnicamente um mar fechado, mas é frequentemente referido como o maior lago do mundo.

2. **Maior lago de água doce por área superficial:** O Lago Superior, localizado na fronteira entre os Estados Unidos e o Canadá, é o maior lago de água doce do mundo em termos de área superficial, com cerca de 82.100 km².

3. **Maior lago por volume de água:** O Lago Baikal, na Rússia,

Quando o assistente precisar consultar o planejador, ele sugerirá uma chamada de função para ask_planner. Quando isso acontecer, uma linha como a seguinte será exibida:

***** Chamada de Função Sugerida: ask_planner *****